In [79]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [80]:
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

Get raw html from page using requests

In [81]:
html = requests.get(link).content

Convert into BeautifulSoup object

In [82]:
soup = BeautifulSoup(html, "html.parser")

Extract table from html

In [83]:
tab=soup.find("table",{"class":"wikitable sortable"})

In [84]:
type(soup)

bs4.BeautifulSoup

In [85]:
type(tab)

bs4.element.Tag

Get headers for dataframe using the th tags

In [86]:
headerCells = tab.find_all("th")
headerVals = []

In [87]:
for x in headerCells:
    print(x.find(text=True))
    headerVals.append(x.find(text=True).strip('\n'))

Postal Code

Borough

Neighborhood



In [88]:
rows = tab.find_all("tr")

In [89]:
df = pd.DataFrame(columns=headerVals)

Loop through the soup object and extract all 'td' tags.  
Append each set of tds in a tr to a separate list.

In [90]:
rowsList = []
row = []
for x in rows:
    row = []
    for y in x.find_all("td"):
        row.append(y.find(text=True).strip('\n'))
    rowsList.append(row)

In [91]:
rowsList = rowsList[1:]

Break up the 2D list into three 1D lists that would act as columns for the dataframe

In [92]:
postalCode = []
borough = []
neighborhood = []
for x in rowsList:
    postalCode.append(x[0])
    borough.append(x[1])
    neighborhood.append(x[2])

Add the lists to the dataframe

In [93]:
df["Postal Code"] = postalCode
df["Borough"] = borough
df["Neighborhood"] = neighborhood

In [94]:
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [95]:
df.tail()

,Postal Code,Borough,Neighborhood
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."
179,M9Z,Not assigned,Not assigned


Drop rows where 'Borough' is 'Not assigned'

In [96]:
for index, row in df.iterrows():
    if row["Borough"] == "Not assigned":
        df = df.drop(index, axis=0)

Check for rows where 'Neighborhood' == 'Not assigned'

No such rows exist

In [97]:
for index, row in df.iterrows():
    if row["Neighborhood"] == "Not assigned":
        print(index)

In [98]:
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [99]:
df.shape

(103, 3)

In [100]:
geo = pd.read_csv("Geospatial_Coordinates.csv")

In [101]:
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [104]:
pc = geo["Postal Code"].values

In [108]:
df["Latitude"] = np.nan
df["Longitude"] = np.nan

Add corresponding latitudes and longitudes to the postal codes

In [124]:
lat = []
lon = []
for index, row in df.iterrows():
    if row["Postal Code"] in pc:
        add = geo.loc[geo["Postal Code"] == row["Postal Code"]]
        lon.append(add["Longitude"].values[0])
        lat.append(add["Latitude"].values[0])
        

In [127]:
df["Longitude"] = lon
df["Latitude"] = lat